In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

Based on fast.ai lesson4-imdb notebook

### Setup

In [1]:
PATH='data/factor/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

SyntaxError: invalid syntax (<ipython-input-1-bafb5629b5dd>, line 5)

### Data Exploration

In [ ]:
trn_files = !ls {TRN}
trn_files[:10]

Example factor file...

In [ ]:
factor_file = !cat {TRN}{trn_files[6]}
factor_file[0]

Tokenize it...

In [ ]:
' '.join(spacy_tok(factor_file[0]))

### Create Model

In [ ]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [ ]:
batch_size=64
back_propp_tt=70

In [ ]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

Save vocab for later...

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

Here are the: # batches; # unique tokens in the vocab; # tokens in the training set; # sentences

In [ ]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

Check mapping of integer id to token...

In [ ]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]

Convert training data into integer IDs of tokens...

In [ ]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

### Train

In [ ]:
em_sz = 2000  # size of each embedding vector (suggested = 200)
nh = 500     # number of hidden activations per layer (suggested = 500)
nl = 3       # number of layers (suggested = 3)

Researchers have found that large amounts of *momentum* (which we'll learn about later) don't work well with these kinds of *RNN* models, so we create a version of the *Adam* optimizer with less momentum than it's default of `0.9`.

In [ ]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [ ]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save_encoder('adam1_enc')

In [ ]:
learner.load_encoder('adam1_enc')

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))